In [ ]:
import os, random, shutil
from pathlib import Path
import cv2
import numpy as np
from PIL import Image

!pip install mtcnn opencv-python

from mtcnn import MTCNN
from google.colab import drive

drive.mount('/content/drive')

##PREPARE DIRECTIONS FOR DATA

In [ ]:
RAW_DIR = Path("/content/drive/MyDrive/final data/Final_data_raw")

EXP_DIR = Path("/content/drive/MyDrive/AI_Project/experiment")
ALIGNED_DIR = EXP_DIR / "aligned_224"
SPLIT_DIR   = EXP_DIR / "split"

TRAIN_DIR = SPLIT_DIR / "train"
VAL_DIR   = SPLIT_DIR / "val"
TEST_DIR  = SPLIT_DIR / "test"

for d in [ALIGNED_DIR, TRAIN_DIR, VAL_DIR, TEST_DIR]:
    d.mkdir(parents=True, exist_ok=True)

##AUGUMENTATION ON ZIAD ABDULLAH (JUST 50 IMAGE BEFOR AND AFTER AUGMENTAION --> 150 TO SOLVE IMAGES SIZE TO PUT ZIAD CLASS IN MODEL TRAINING )

In [ ]:
from PIL import Image, ImageEnhance
import random
import shutil

MY_NAME = "ziad abdullah"
MY_FOLDER = RAW_DIR / MY_NAME

AUG_PER_IMAGE = 2

images = list(MY_FOLDER.glob("*"))

for img_path in images:
    for i in range(AUG_PER_IMAGE):
        img = Image.open(img_path).convert("RGB")

        # 1. random flip
        if random.random() < 0.5:
            img = img.transpose(Image.FLIP_LEFT_RIGHT)

        # 2. random rotation
        if random.random() < 0.5:
            angle = random.randint(-10, 10)
            img = img.rotate(angle)

        # 3. brightness/contrast
        if random.random() < 0.5:
            enhancer = ImageEnhance.Brightness(img)
            img = enhancer.enhance(random.uniform(0.8, 1.3))

        if random.random() < 0.5:
            enhancer = ImageEnhance.Contrast(img)
            img = enhancer.enhance(random.uniform(0.8, 1.3))

        new_name = img_path.stem + f"_aug{i}" + img_path.suffix
        save_path = MY_FOLDER / new_name
        img.save(save_path)

print("🔥 Personal augmentation complete!")


##test

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

img_path = MY_FOLDER / "WhatsApp Image 2025-12-19 at 10.28.32 PM(1)_aug1.jpeg"
img_path1 = MY_FOLDER / "WhatsApp Image 2025-12-19 at 10.28.32 PM(1)_aug0.jpeg"

img = Image.open(img_path)
img1 = Image.open(img_path1)

plt.imshow(img)
plt.imshow(img1)

plt.axis('off')
plt.show()


##DO FACE ALIGNED ON ZIAD ABDULLAH

In [ ]:
from pathlib import Path
import cv2
from mtcnn import MTCNN
from PIL import Image

RAW_DIR = Path("/content/drive/MyDrive/final data/Final_data_raw")
ALIGNED_DIR = Path("/content/drive/MyDrive/AI_Project/experiment/aligned_224")

MY_NAME = "ziad abdullah"
MY_FOLDER = RAW_DIR / MY_NAME
OUT_FOLDER = ALIGNED_DIR / MY_NAME

OUT_FOLDER.mkdir(parents=True, exist_ok=True)

IMG_SIZE = 224


detector = MTCNN()


images = list(MY_FOLDER.glob("*"))
for img_path in images:
    img = cv2.imread(str(img_path))
    if img is None:
        continue

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    faces = detector.detect_faces(img_rgb)
    if len(faces) == 0:
        continue

    face = max(faces, key=lambda f: f['box'][2] * f['box'][3])
    x, y, w, h = face['box']
    x, y = max(0, x), max(0, y)

    face_img = img[y:y+h, x:x+w]
    if face_img.size == 0:
        continue

    face_img = cv2.resize(face_img, (IMG_SIZE, IMG_SIZE))

    cv2.imwrite(str(OUT_FOLDER / img_path.name), face_img)

print(f"✅ Face alignment done for {MY_NAME}, saved in {OUT_FOLDER}")


##TEST AFTER FACE ALIGNED

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import os

image_dir = "/content/drive/MyDrive/AI_Project/experiment/aligned_224/ziad abdullah"
image_files = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]

if image_files:
    img_path = os.path.join(image_dir, image_files[17]) # Take the first image found
else:
    print(f"No image files found in {image_dir}")
    img_path = None # Handle case where no images are found

if img_path:
    img = Image.open(img_path)

    plt.imshow(img)

    plt.axis('off')
    plt.show()
else:
    print("Could not display image as no valid path was found.")


##DO FACE ALIGNED ON OTHER IMAGES (WITHOUT ZIAD )

In [ ]:
detector = MTCNN()

persons = sorted([p for p in RAW_DIR.iterdir() if p.is_dir()])
persons = persons[:MAX_CLASSES]

print("Target classes:", len(persons))

for person_folder in persons:
    person_out = ALIGNED_DIR / person_folder.name
    person_out.mkdir(parents=True, exist_ok=True)

    images = list(person_folder.glob("*"))[:MAX_IMAGES_PER_CLASS]

    for img_path in images:
        img = cv2.imread(str(img_path))
        if img is None:
            continue

        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        faces = detector.detect_faces(img_rgb)
        if len(faces) == 0:
            continue

        face = max(faces, key=lambda f: f['box'][2] * f['box'][3])
        x, y, w, h = face['box']
        x, y = max(0, x), max(0, y)

        face_img = img[y:y+h, x:x+w]
        if face_img.size == 0:
            continue

        face_img = cv2.resize(face_img, (IMG_SIZE, IMG_SIZE))
        cv2.imwrite(str(person_out / img_path.name), face_img)

print("✅ Face alignment finished")

In [ ]:
valid_persons = []

for person in sorted(ALIGNED_DIR.iterdir()):
    if not person.is_dir():
        continue

    images = list(person.glob("*"))
    if len(images) >= MIN_IMAGES_AFTER_ALIGNMENT:
        valid_persons.append(person)

my_folder = ALIGNED_DIR / "ziad abdullah"
if my_folder.exists() and my_folder not in valid_persons:
    valid_persons.append(my_folder)

print("Valid classes after alignment:", len(valid_persons))

In [ ]:
random.seed(42)

for person in valid_persons:
    images = list(person.glob("*"))
    random.shuffle(images)

    n = len(images)
    n_train = int(0.7 * n)
    n_val   = int(0.1 * n)

    train_imgs = images[:n_train]
    val_imgs   = images[n_train:n_train+n_val]
    test_imgs  = images[n_train+n_val:]

    for split_dir, imgs in zip(
        [TRAIN_DIR, VAL_DIR, TEST_DIR],
        [train_imgs, val_imgs, test_imgs]
    ):
        dest = split_dir / person.name
        dest.mkdir(parents=True, exist_ok=True)

        for img in imgs:
            shutil.copy2(img, dest / img.name)

print("✅ Split 70/20/10 completed")
